In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
sns.set(color_codes=True)

/Users/greg/anaconda/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [19]:
data = pd.read_csv('./RawDatafromJim_Complete.csv', usecols=[0,1,3,6,7,16])
data = data.apply(lambda x: pd.to_numeric(x, errors='ignore'))
data["weight"]=1

In [20]:
data

,cited_case,citing_case,citing_opinion_type,citing_case_year,cited_case_year,cited_usid,weight
0,1796 U.S. LEXIS 409,1792 U.S. LEXIS 587,0,1792,1796,3 U.S. 320,1
1,1793 U.S. LEXIS 249,1793 U.S. LEXIS 247,0,1793,1793,2 U.S. 419,1
2,1796 U.S. LEXIS 409,1793 U.S. LEXIS 247,0,1793,1796,3 U.S. 320,1
3,1796 U.S. LEXIS 409,1793 U.S. LEXIS 248,0,1793,1796,3 U.S. 320,1
4,1798 U.S. LEXIS 145,1793 U.S. LEXIS 249,0,1793,1798,3 U.S. 378,1
5,1792 U.S. LEXIS 590,1793 U.S. LEXIS 249,0,1793,1792,2 U.S. 402,1
6,1793 U.S. LEXIS 248,1793 U.S. LEXIS 249,0,1793,1793,2 U.S. 415,1
7,1795 U.S. LEXIS 329,1795 U.S. LEXIS 327,0,1795,1795,3 U.S. 54,1
8,1795 U.S. LEXIS 326,1795 U.S. LEXIS 329,0,1795,1795,3 U.S. 17,1
9,1794 U.S. LEXIS 103,1795 U.S. LEXIS 329,0,1795,1794,3 U.S. 6,1


In [21]:
def case_data_for_year(calc_year):
    year_data = data.loc[data['citing_case_year'] <= calc_year]
    citing_year = year_data.filter(['citing_case','citing_case_year'], axis=1)
    citing_year = citing_year.rename(columns={'citing_case':'case', 'citing_case_year': 'year'})
    cited_year = year_data.filter(['cited_case','cited_case_year'], axis=1)
    cited_year = cited_year.rename(columns={'cited_case':'case', 'cited_case_year': 'year'})
    case_year = citing_year.append(cited_year)
    case_year_dict = {}
    for row in case_year.iterrows():
        case_year_dict[row[1].case] = row[1].year
    
    G = nx.from_pandas_dataframe(year_data,"citing_case","cited_case",["weight"],nx.DiGraph())
    h,a = nx.hits(G,1000,normalized=False)
    if(len(h) > 0 and len(a) > 0):
        d = []
        for key in h:
            d.append({'calc_year': calc_year, 'case':key, 'hub_raw': h[key], 'auth_raw': a[key], 'case_year': case_year_dict[key]})
        year_case_scores = pd.DataFrame(d)
        year_case_scores['hub_percentile'] = year_case_scores.rank(pct=True)['hub_raw']
        year_case_scores['auth_percentile'] = year_case_scores.rank(pct=True)['auth_raw']
    else:
        return None
    return year_case_scores


In [22]:
years_to_analyize = data.cited_case_year.unique()
years_to_analyize.sort()
years_to_analyize = years_to_analyize[0:10]
years_to_analyize = [2002]

In [23]:
case_data = pd.DataFrame()
for year in years_to_analyize:
    cdy = case_data_for_year(year)
    if(cdy is None):
        continue
    else:
        case_data = case_data.append(cdy)

In [33]:
case_data.sort_index(by=['hub_percentile'], ascending=[False], inplace=True)
case_data

/Users/greg/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if __name__ == '__main__':


,auth_raw,calc_year,case,case_year,hub_raw,hub_percentile,auth_percentile
21454,1.730155e-01,2002,1972 U.S. LEXIS 132,1972,1.000000e+00,1.000000,0.991769
20452,4.633666e-01,2002,1965 U.S. LEXIS 2282,1965,9.436316e-01,0.999961,0.998785
9402,1.337898e-01,2002,1961 U.S. LEXIS 1934,1961,8.829462e-01,0.999922,0.987144
19551,5.143040e-01,2002,1976 U.S. LEXIS 16,1976,8.755859e-01,0.999882,0.999216
23822,1.780586e-01,2002,1973 U.S. LEXIS 19,1973,8.287130e-01,0.999843,0.992239
12637,1.674500e-01,2002,1973 U.S. LEXIS 4,1973,8.153684e-01,0.999804,0.991103
11073,5.004981e-02,2002,1971 U.S. LEXIS 107,1971,8.141868e-01,0.999765,0.953004
6259,2.541584e-01,2002,1978 U.S. LEXIS 83,1978,8.096573e-01,0.999726,0.996080
23638,2.359860e-01,2002,1976 U.S. LEXIS 3,1976,8.046126e-01,0.999686,0.995101
12430,2.698834e-01,2002,1951 U.S. LEXIS 2407,1951,7.351862e-01,0.999647,0.996472


In [28]:
case_data["hub_percentile"].dtype

dtype('float64')

In [16]:
testdata = pd.read_csv('./RawDatafromJim_Complete.csv')

/Users/greg/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,9,10,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
testdata

,cited_case,citing_case,citing_court,citing_opinion_type,shepards,year_correct,citing_case_year,cited_case_year,us_spaeth_cited_case,led_spaeth_cited_case,docket_spaeth_cited_case,term_spaeth_cited_case,us_spaeth_citing_case,led_spaeth_citing_case,docket_spaeth_citing_case,term_spaeth_citing_case,cited_usid
0,1796 U.S. LEXIS 409,1792 U.S. LEXIS 587,U.S. SUPREME COURT,0,cite,1.0,1792,1796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 U.S. 320
1,1793 U.S. LEXIS 249,1793 U.S. LEXIS 247,U.S. SUPREME COURT,0,cite,NaN,1793,1793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 U.S. 419
2,1796 U.S. LEXIS 409,1793 U.S. LEXIS 247,U.S. SUPREME COURT,0,cite,1.0,1793,1796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 U.S. 320
3,1796 U.S. LEXIS 409,1793 U.S. LEXIS 248,U.S. SUPREME COURT,0,cite,1.0,1793,1796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 U.S. 320
4,1798 U.S. LEXIS 145,1793 U.S. LEXIS 249,U.S. SUPREME COURT,0,cite,1.0,1793,1798,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 U.S. 378
5,1792 U.S. LEXIS 590,1793 U.S. LEXIS 249,U.S. SUPREME COURT,0,cite,NaN,1793,1792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 U.S. 402
6,1793 U.S. LEXIS 248,1793 U.S. LEXIS 249,U.S. SUPREME COURT,0,cite,NaN,1793,1793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 U.S. 415
7,1795 U.S. LEXIS 329,1795 U.S. LEXIS 327,U.S. SUPREME COURT,0,cite,NaN,1795,1795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 U.S. 54
8,1795 U.S. LEXIS 326,1795 U.S. LEXIS 329,U.S. SUPREME COURT,0,cite,NaN,1795,1795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 U.S. 17
9,1794 U.S. LEXIS 103,1795 U.S. LEXIS 329,U.S. SUPREME COURT,0,cite,NaN,1795,1794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 U.S. 6
